In [1]:
import AlgoCrac as ac

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from abc import ABC, abstractmethod
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import partial
import matplotlib.pyplot as plt
import datetime
#from indicators import moving_average, exp_moving_average, vwap
#from Signals import *
from AlgoCrac import moving_average, exp_moving_average, vwap
from AlgoCrac.Signals.Signals import *
from AlgoCrac.Signals.utils import *
from AlgoCrac.Strategy.Strategy import *


from tqdm import tqdm

In [3]:



timeframe = "5m"
#sym = ['WIX','MMM','GOOGL','TSLA']
sym = ["BDN",'MMM']
sym = ['MMM']
data = ac.get_data(sym, start = "2024-06-12",end="2024-06-13",interval = timeframe)

[*********************100%***********************]  1 of 1 completed


In [4]:
data

,Open,High,Low,Close,Adj Close,Volume
,MMM,MMM,MMM,MMM,MMM,MMM
2024-06-12 09:30:00,102.620003,103.430000,102.510002,103.160004,103.160004,278959
2024-06-12 09:35:00,103.180000,103.360001,102.840103,103.235001,103.235001,102129
2024-06-12 09:40:00,103.250000,103.620003,103.220001,103.555000,103.555000,80842
2024-06-12 09:45:00,103.559998,103.680000,103.129997,103.449997,103.449997,96051
2024-06-12 09:50:00,103.400002,103.419998,102.870003,102.894997,102.894997,84213
...,...,...,...,...,...,...
2024-06-12 15:35:00,100.879997,100.959999,100.809998,100.816902,100.816902,51592
2024-06-12 15:40:00,100.820000,100.930000,100.790001,100.830002,100.830002,44162
2024-06-12 15:45:00,100.808296,101.080002,100.769997,101.039001,101.039001,59978


Her is a case where a strategy fails, this is due to intraday days with the signal's periods being larger than the number of timesteps within a single trading day.

In [5]:

period = 120
indicator =  "SMA"
buy_signal = Crossover(period, indicator = indicator, on = "Close", invert = False, eps = 0.2)
sell_signal = Crossover(period, indicator = indicator, on = "Close", invert = True, eps = 0.2)

st = Strategy(buy_signal, sell_signal,
              timeframe=timeframe,
                trade_allocation = 1000,
                  allocation = 10000,
                    negative_allocation= False,
                      side = 'long')

Error: period of the signal is too large for intraday strategy, strategy is not initialized


In [6]:

range = Range(max_val = 95,
                         min_val = 85,
                        metric = "last",
                        in_range = True, 
                        on = "Close", # Open, High, Low, Close, Volume
                        invert = False,
                        )



signal_list = [range]
#signal_list = [vwap_crossover]
#signal_list = [v_crossover]

#signal_list = [ ema_crossover]
ma_confl = Confluence(signals_list = signal_list, max_signals = 3, invert = False)

buy_signal = ma_confl
buy_signal.plot_signals(data, signal_type = 'con')

sell_signal = Confluence(signals_list = signal_list, max_signals = 3, invert = True)

sell_signal.plot_signals(data, signal_type = 'con')

plot intraday data


plot intraday data


In [7]:
st = Strategy(buy_signal, sell_signal,
              timeframe=timeframe,
                trade_allocation = 1000,
                  allocation = 10000,
                    negative_allocation= False,
                      side = 'long')

## Backtest

In [8]:
st.reset()
bt = Backtest(st, data, fast = False, include_commission= True)
a = bt.run()  
bt.summary() 

100%|██████████| 74/74 [00:00<00:00, 404.52it/s]

Total time strategy update:  0.16695880889892578
Total time status update:  0.01202702522277832


,allocation,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,win rate
closed,0,0,0,0,0,0,0,0,0,0,0,0
all,0,0,0,0,0,0,0,0,0,0,0,0


* -all- refers to the status at any given time (open and closed trades)
* -closed- refers only to closed trades

bt.status

In [9]:
bt.strategy.closed_trades


,reward


## Fast backtest

In [10]:

st.reset()
bt = Backtest(st, data, fast = True, include_commission= True)
a = bt.run()  
bt.summary() 

[datetime.date(2024, 6, 12)]
Total time strategy update:  0.004021644592285156
Total time status update:  0.0


,allocation,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,win rate
closed,0,0,0,0,0,0,0,0,0,0,0,0
all,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
symbol_summary = bt.strategy.closed_trades[['symbol','reward']].groupby('symbol').sum().copy()
symbol_summary['reward (average)'] = bt.strategy.closed_trades[['symbol','reward']].groupby('symbol').mean()
symbol_summary

KeyError: "['symbol'] not in index"

In [ ]:
bt.strategy.closed_trades

,entry time,entry date,exit time,exit date,entry price,exit price,symbol,reward,position,commission,market value,id
0,13:20:00,2024-06-12,13:25:00,2024-06-12,101.510002,101.529999,MMM,0.179970,9,1.5,913.769989,0
1,13:35:00,2024-06-12,13:45:00,2024-06-12,101.570000,101.425003,MMM,-1.304970,9,1.5,912.825027,1
2,15:05:00,2024-06-12,15:10:00,2024-06-12,101.105003,101.169998,MMM,0.584953,9,1.5,910.529984,2
3,15:50:00,2024-06-12,15:55:00,2024-06-12,101.040001,101.360001,MMM,2.879997,9,1.5,912.240005,3


In [ ]:
bt.strategy.open_trades

,reward


In [ ]:
pd.concat([bt.strategy.closed_trades.head(),bt.strategy.open_trades.head()], ignore_index=True)

,entry time,entry date,exit time,exit date,entry price,exit price,symbol,reward,position,commission,market value,id
0,13:20:00,2024-06-12,13:25:00,2024-06-12,101.510002,101.529999,MMM,0.179970,9.0,1.5,913.769989,0.0
1,13:35:00,2024-06-12,13:45:00,2024-06-12,101.570000,101.425003,MMM,-1.304970,9.0,1.5,912.825027,1.0
2,15:05:00,2024-06-12,15:10:00,2024-06-12,101.105003,101.169998,MMM,0.584953,9.0,1.5,910.529984,2.0
3,15:50:00,2024-06-12,15:55:00,2024-06-12,101.040001,101.360001,MMM,2.879997,9.0,1.5,912.240005,3.0
